In [1]:
#Los 3 primeros imports son para llamar a la API de ChatGPT
from dotenv import load_dotenv
import os #provides ways to access the Operating System and allows us to read the environment variables
import openai

import requests #Este import es para llamar a la API del tiempo

import pandas as pd #Este import es para cargar el CSV de latitud y longitud de todos los pueblos de españa

from haversine import haversine, Unit #Estos import son para calcular las distancias de lat y long
from math import radians, sin, cos, sqrt, atan2


import streamlit as st #Es para poder crear la APP

In [2]:
#Referencias streamlit https://seraph13.medium.com/tutorial-introductorio-de-streamlit-be78f225b434

st.title("CampToGo")

2023-07-15 09:56:46.274 
  command:

    streamlit run C:\Users\lourdes.ruiz\miniconda3\envs\project_new_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [3]:
st.header("Welcome user")
st.subheader("This is the APP that helps you to plan your last minute road trip in Campervan starting from Madrid")

DeltaGenerator()

In [4]:
#Llamo a la API de chatGPT aunque ahora he decidido no empezar usandola

load_dotenv()
my_secret_key = os.getenv("my_openai_key")

In [5]:
#Llamo a la contraseña que tengo guardada en .env

openai.api_key = my_secret_key

In [6]:
#Le doy la bienvenida al usuario

print("Welcome to the APP that helps you to plan your last minute road trip in Campervan starting from Madrid")

Welcome to the APP that helps you to plan your last minute road trip in Campervan starting from Madrid


In [7]:
#Le pregunto al usuario cuantos días va a viajar ya que es la base de mi elección de itinerario

days = int(input("Please enter how many days are you going to travel: "))

Please enter how many days are you going to travel: 1


In [8]:
#Creo un diccionario en base al radio de KM para usarlo como referencia para saber el tiempo

dictionary = {
    'N': ['Riaza', 'Aranda de Duero', 'Burgos', 'Santander', 'Santander'],
    'NO': ['Segovia', 'Zamora', 'Ponferrada', 'Lugo', 'A Coruña'],
    'O': ['Ávila', 'Salamanca', 'Ciudad Rodrigo', 'Ciudad Rodrigo', 'Ciudad Rodrigo'],
    'SO': ['Talavera de la Reina', 'Plasencia', 'Mérida', 'Sevilla', 'Huelva'],
    'S': ['Toledo', 'Ciudad Real', 'Jaén', 'Granada', 'Málaga'],
    'SE': ['Saelices', 'Albacete', 'Almansa', 'Murcia', 'Almería'],
    'E': ['Sacedón', 'Cuenca', 'Valencia', 'Jávea', 'Alicante'],
    'NE': ['Sigüenza', 'Calatayud', 'Zaragoza', 'Huesca', 'Andorra']}

In [9]:
#Creo una función para que me calcule la posición en el diccionario en base a los días que va a viajar el usuario
# Pero si introduce más de 5 días le devuelvo la última posición del diccionario independientemente de los días

def days_limited(days: int):
    
    d = 0
    
    if days >=8:
        d = 4
    elif days == 7 or days >= 2: 
        d = int(days - 2)
    else:
        d = 0
        
    return d

In [10]:
days_lim = days_limited(days)
days_lim

0

In [11]:
#Creo una función para que me genere automaticamente las ubicaciones en base a los días de viaje

def get_places(dictionary, d):
    values = {}
    for key, value_list in dictionary.items():
        if d < len(value_list):
            values[key] = value_list[d]
    return values

In [12]:
#Reviso que me devuelva el resultado correcto https://www.weatherapi.com/docs/

result = get_places(dictionary, days_lim)
result

{'N': 'Riaza',
 'NO': 'Segovia',
 'O': 'Ávila',
 'SO': 'Talavera de la Reina',
 'S': 'Toledo',
 'SE': 'Saelices',
 'E': 'Sacedón',
 'NE': 'Sigüenza'}

In [13]:
#Creo todas las variables para llamar a la API del tiempo menos "q" que representa el lugar 
#de dónde quiero ver la previsión del tiempo
load_dotenv()
url = 'http://api.weatherapi.com/v1'
forecast = '/forecast.json?'
pre = 'key='
key =os.getenv("my_weather_token")
wdays = '&days=1'
final = '&aqi=no&alerts=no'

#print(url + forecast + pre + key + wdays + final)

In [14]:
#Ahora creo una variable "q" para cada punto cardinal

q_var = '&q='
norte = q_var + result['N']
noroeste = q_var + result['NO']
oeste = q_var + result['O']
suroeste = q_var + result['SO']
sur = q_var + result['S']
sureste = q_var + result['SE']
este = q_var + result['E']
noreste = q_var + result['NE']

In [15]:
#Creo una llamada a la API para cada lugar

check_rain_north = requests.get(url + forecast + pre + key + norte + wdays + final)
check_rain_nortwest = requests.get(url + forecast + pre + key + noroeste + wdays + final)
check_rain_west = requests.get(url + forecast + pre + key + oeste + wdays + final)
check_rain_southwest = requests.get(url + forecast + pre + key + suroeste + wdays + final)
check_rain_south = requests.get(url + forecast + pre + key + sur + wdays + final)
check_rain_southeast = requests.get(url + forecast + pre + key + sureste + wdays + final)
check_rain_east = requests.get(url + forecast + pre + key + este + wdays + final)
check_rain_northeast = requests.get(url + forecast + pre + key + noreste + wdays + final)

#check_rain_southeast.status_code

In [16]:
check_rain_north_result = check_rain_north.json()
check_rain_nortwest_result = check_rain_nortwest.json()
check_rain_west_result = check_rain_west.json()
check_rain_southwest_result = check_rain_southwest.json() 
check_rain_south_result = check_rain_south.json() 
check_rain_southeast_result = check_rain_southeast.json()
check_rain_east_result = check_rain_east.json()
check_rain_northeast_result = check_rain_northeast.json()

In [17]:
print("Awesome! I suggest to travel to the following directions as the others could rain tomorrow")

#North
if check_rain_north_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('North')
    
#Northwest
if check_rain_nortwest_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Northwest')

#West
if check_rain_west_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('West')
    
#Southwest
if check_rain_southwest_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Southwest')
    
#South
if check_rain_south_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('South')

#Southeast
if check_rain_southeast_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Southeast')

#East
if check_rain_east_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('East')
    
#Northeast
if check_rain_northeast_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Northeast')

Awesome! I suggest to travel to the following directions as the others could rain tomorrow
North
Northwest
West
Southwest
Southeast
East
Northeast


In [18]:
#Ahora le pido al usuario que elija el punto cardinal

direction = input('Choose one: ')

Choose one: southeast


In [19]:
#Cargo el dataset de municipios para tener las coordenadas

municipios = pd.read_csv('./data/pueblos_bonitos_punto.csv')
municipios[:8]

,Pueblo,Provincia,Longitud,Latitud,Datos de Interés
0,Alcalá del Júcar,Albacete,-1.429327,39.193202,D.O. Pueblo Bonito
1,Almagro,Ciudad Real,-3.711559,38.889841,D.O. Pueblo Bonito
2,Almonaster la Real,Huelva,-6.787200,37.872631,D.O. Pueblo Bonito
3,Anento,Zaragoza,-1.333285,41.069730,D.O. Pueblo Bonito
4,Ansó,Huesca,-0.828883,42.754663,D.O. Pueblo Bonito
5,Atienza,Guadalajara,-2.871087,41.200040,D.O. Pueblo Bonito
6,Ayllón,Segovia,-3.376887,41.420808,D.O. Pueblo Bonito
7,Baños de la Encina,Jaén,-3.774262,38.171199,D.O. Pueblo Bonito


In [20]:
#Cargo el dataset de las coordenadas de mis puntos cardinales para calcular las distancias

p_cardinales = pd.read_csv('./data/puntos_cardinales_punto.csv')
p_cardinales

,Pueblo,Provincia,Longitud,Latitud
0,Albacete,Albacete,-1.855747,38.995881
1,Almansa,Albacete,-1.094147,38.869688
2,Andorra,Teruel,-0.443719,40.975094
3,Aranda de Duero,Burgos,-3.689243,41.671551
4,Burgos,Burgos,-3.704198,42.341130
5,Calatayud,Zaragoza,-1.644726,41.354319
6,Ciudad Real,Ciudad Real,-3.931320,38.986518
7,Ciudad Rodrigo,Salamanca,-6.534158,40.597309
8,Cuenca,Cuenca,-2.131523,40.076538
9,Granada,Granada,-3.600019,37.176419


In [21]:
#Ahora hago match entre la dirección que ha elegido el usuario y las opciones que tengo

def user_direction(direction, result):
    if direction == 'north' or direction == 'N' or direction == 'North':
        return result['N']
    elif direction == 'northwest' or direction == 'NW' or direction == 'Northwest' or direction == 'NorthWest':
        return result['NO']
    elif direction == 'west' or direction == 'West' or direction == 'W':
        return result['O']
    elif direction == 'southwest' or direction == 'Southwest' or direction == 'SouthWest' or direction == 'SW':
        return result['SO']
    elif direction == 'south' or direction == direction == 'S' or direction == 'South':
        return result['S']
    elif direction == 'southeast' or direction == 'Southeast' or direction == 'SouthEast' or direction == 'SE':
        return result['SE']
    elif direction == 'east' or direction == 'East' or direction == 'E':
        return result['E']
    elif direction == 'northeast' or direction == 'Northeast' or direction == 'NorthEast' or direction == 'NE':
        return result['NE']
    else:
        return 'I am not able to find the direction'

In [22]:
#Guardo el resultado en un diccionario

result = {'N': result['N'], 'NO': result['NO'], 'O': result['O'], 'NE': result['NE'],
          'S': result['S'], 'SO': result['SO'], 'SE': result['SE'], 'E': result['E']}

In [23]:
output = user_direction(direction, result)
output

'Saelices'

In [24]:
#Guardo en las variables de latitud y longitud desde dónde quiero crear el radio de los pueblos que recomendar

row = p_cardinales[p_cardinales['Pueblo'] == output]
row

,Pueblo,Provincia,Longitud,Latitud
36,Saelices,Cuenca,-2.805578,39.920977


In [25]:
idx = row.index[row['Pueblo'] == output][0]
idx

36

In [26]:
lat_origen = row['Latitud'][idx]
long_origen = row['Longitud'][idx]

In [27]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate distance
    distance = R * c
    return distance

In [28]:
municipios["Distance"] = municipios.apply(lambda row: haversine(row["Latitud"], row["Longitud"], lat_origen, long_origen), axis=1)
municipios

,Pueblo,Provincia,Longitud,Latitud,Datos de Interés,Distance
0,Alcalá del Júcar,Albacete,-1.429327,39.193202,D.O. Pueblo Bonito,143.069120
1,Almagro,Ciudad Real,-3.711559,38.889841,D.O. Pueblo Bonito,138.580992
2,Almonaster la Real,Huelva,-6.787200,37.872631,D.O. Pueblo Bonito,412.978009
3,Anento,Zaragoza,-1.333285,41.069730,D.O. Pueblo Bonito,178.363510
4,Ansó,Huesca,-0.828883,42.754663,D.O. Pueblo Bonito,355.665863
...,...,...,...,...,...,...
69,Viniegra de Arriba,La Rioja,-2.834787,42.095493,D.O. Pueblo Bonito,241.807562
70,Vinuesa,Soria,-2.762725,41.911792,D.O. Pueblo Bonito,221.397729
71,Yanguas,Soria,-2.339304,42.101758,D.O. Pueblo Bonito,245.626239
72,Zahara,Cádiz,-5.390317,36.840277,D.O. Pueblo Bonito,409.961487


In [29]:
top_3 = municipios.sort_values(by='Distance').reset_index(drop=True)[:3]
top_3

,Pueblo,Provincia,Longitud,Latitud,Datos de Interés,Distance
0,Pastrana,Guadalajara,-2.924589,40.416315,D.O. Pueblo Bonito,55.999636
1,Chinchón,Madrid,-3.422306,40.141269,D.O. Pueblo Bonito,57.941415
2,Nuevo Baztán,Madrid,-3.243131,40.366149,D.O. Pueblo Bonito,61.915931


In [30]:
one = top_3.iloc[0, top_3.columns.get_loc('Pueblo')]
two = top_3.iloc[1, top_3.columns.get_loc('Pueblo')]
three = top_3.iloc[2, top_3.columns.get_loc('Pueblo')]

In [31]:
print('Cool! I recommend you to visit these 3 villages selected as Pueblos Bonitos de España: ') 
print(one)
print(two)
print(three)

Cool! I recommend you to visit these 3 villages selected as Pueblos Bonitos de España: 
Pastrana
Chinchón
Nuevo Baztán


In [32]:
print('Now, let me give you some tips. Find below what to visit, where you can park to sleep and popular routes for trekking')

Now, let me give you some tips. Find below what to visit, where you can park to sleep and popular routes for trekking


In [33]:
def generate_monuments_prompt(place):
    chat_prompt = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"List the top places to visit in {place} but\
        don't explain them and do not give me a introduction , just the answer with\
        the list. After that, let me know where I should park to sleep in a campervan.\
        Last, list me the 3 most common tracking routes and show me the amount of kilometers\
        and the difficulty level rated by easy, medium or high."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=chat_prompt
    )

    return response.choices[0].message.content.strip()

In [34]:
place = one 

In [35]:
response_text = generate_monuments_prompt(place)
print(one)
print(response_text)

Pastrana
Top places to visit in Pastrana:
1. Pastrana Castle
2. Plaza de la Hora
3. Iglesia Colegiata de Pastrana
4. Museo del Territorio Histórico de Pastrana
5. Hospital del Cardenal
6. Convento del Carmen
7. Museo Parroquial de la Colegiata
8. Antiguas Caballerizas de Pastrana

Parking for sleeping in a campervan:
There are a few options for campervan parking in Pastrana. Some popular spots include:
1. Parking lot near Plaza de la Hora
2. Parking area by the Pastrana Castle
3. A designated camping site outside the town

Top 3 tracking routes:
1. Route: Bosque de Pastrana
   Distance: 6.5 km
   Difficulty level: Easy

2. Route: Ruta del Tajo de las Carpinas
   Distance: 12 km
   Difficulty level: Medium

3. Route: Sendero de las Fuentes de la Alcarria
   Distance: 18 km
   Difficulty level: High


In [36]:
place = two

In [37]:
response_text = generate_monuments_prompt(place)
print(two)
print(response_text)

Chinchón
Top places to visit in Chinchón:
1. Plaza Mayor
2. Castilo de Chinchón
3. Church of Nuestra Señora de la Asunción
4. Casa de la Cadena
5. Convent of the Clarisas
6. Lope de Vega Theatre
7. Iglesia San Esteban
8. Casa del Regidor
9. Museo Etnológico

For parking and sleeping in a campervan, you can park at:
- Parking Lot "Plaza Mayor" in Chinchón

The three most common tracking routes in Chinchón are:
1. Chinchón Circular Route
   - Distance: 8 km
   - Difficulty: Easy

2. Chinchón to Colmenar de Oreja Route
   - Distance: 12 km
   - Difficulty: Medium

3. Chinchón to Carabaña Route
   - Distance: 20 km
   - Difficulty: High


In [38]:
place = three

In [39]:
response_text = generate_monuments_prompt(place)
print(three)
print(response_text)

Nuevo Baztán
Top places to visit in Nuevo Baztán:
1. Palacio de Goyeneche
2. Iglesia de San Francisco Javier
3. Real Fábrica de Paños
4. Plaza de la Iglesia
5. Casa de las Infantas

Parking for campervans in Nuevo Baztán: There is a designated area for campervan parking near the Plaza de la Iglesia.

Top 3 tracking routes in Nuevo Baztán:
1. Route 1: Sendero del Arroyo Berro
   - Distance: 5 kilometers
   - Difficulty: Easy

2. Route 2: Sendero de la Dehesa de las Cuevas
   - Distance: 8 kilometers
   - Difficulty: Medium

3. Route 3: Sendero del Monte de los Alamillos
   - Distance: 12 kilometers
   - Difficulty: High


In [40]:
print('Enjoy your trip!') 

Enjoy your trip!
